In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/auto-pian

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1jn_llb-3OnamAo89wMXsdDKi__iyjswV/auto-pian


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os

In [3]:
directory_path = 'PianoFingeringDataset_v1.2/PianoFingeringDataset_v1.2/FingeringFiles/'

# Loads in ONE big CONCATENATED dataset from ALL dataframes
# my_dfs = []

# for filename in os.listdir(directory_path):
#     file_path = os.path.join(directory_path, filename)
#     if os.path.isfile(file_path):
#         df = pd.read_table(file_path, sep="\t", skiprows=1, names=["noteID", "onset_time", "offset_time", "spelled_pitch", "onset_velocity", "offset_velocity", "channel", "finger_number"])
#         my_dfs.append(df)
# #         print(df)

# big_df = pd.concat(my_dfs)
# print(big_df)

# load in just the fingering for 014-3: Mozart Piano Sonata K 330 in C major, 2nd mov.
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path) and filename == os.listdir(directory_path)[0]:
        df = pd.read_table(file_path, sep="\t", skiprows=1, names=["noteID", "onset_time", "offset_time", "spelled_pitch", "onset_velocity", "offset_velocity", "channel", "finger_number"])
        print(filename)
        print(df.head(10))



117-1_fingering.txt
   noteID  onset_time  offset_time spelled_pitch  onset_velocity  \
0       0     1.50000      1.66650            A4              62   
1       1     1.50000      1.66650            A5              62   
2       2     1.66650      1.83349           Bb4              62   
3       3     1.66650      1.83349           Bb5              62   
4       4     1.83349      2.00000            A4              62   
5       5     1.83349      2.00000            A5              62   
6       6     2.00000      2.66650           G#5              62   
7       7     2.00000      3.49999           G#4              62   
8       8     2.00000      2.16650            D2              62   
9       9     2.16650      2.33349           Bb2              62   

   offset_velocity  channel  finger_number  
0               80        0              1  
1               80        0              5  
2               80        0              1  
3               80        0              4  
4     

In [4]:
df.isna().sum()

,0
noteID,0
onset_time,0
offset_time,0
spelled_pitch,0
onset_velocity,0
offset_velocity,0
channel,0
finger_number,0


# Idea 1: simple linear regression to predict finger_number

In [5]:
spelled_pitch_values = set()

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        df = pd.read_table(file_path, sep="\t", skiprows=1, names=["noteID", "onset_time", "offset_time", "spelled_pitch", "onset_velocity", "offset_velocity", "channel", "finger_number"])
        spelled_pitch_values.update(df['spelled_pitch'].unique())



# convert "spelled pitch" field to a number: create the mapping in the first place
spelled_pitch_values = sorted(spelled_pitch_values)

pitch_to_int_mapping = {p:i for i, p in enumerate(spelled_pitch_values)}

# print(len(spelled_pitch_values))
# print(len(pitch_to_int_mapping))
print(pitch_to_int_mapping)

{'A1': 0, 'A2': 1, 'A3': 2, 'A4': 3, 'A5': 4, 'A6': 5, 'B1': 6, 'B2': 7, 'B3': 8, 'B4': 9, 'B5': 10, 'B6': 11, 'Bb1': 12, 'Bb2': 13, 'Bb3': 14, 'Bb4': 15, 'Bb5': 16, 'Bb6': 17, 'C#1': 18, 'C#2': 19, 'C#3': 20, 'C#4': 21, 'C#5': 22, 'C#6': 23, 'C#7': 24, 'C1': 25, 'C2': 26, 'C3': 27, 'C4': 28, 'C5': 29, 'C6': 30, 'C7': 31, 'D1': 32, 'D2': 33, 'D3': 34, 'D4': 35, 'D5': 36, 'D6': 37, 'D7': 38, 'E1': 39, 'E2': 40, 'E3': 41, 'E4': 42, 'E5': 43, 'E6': 44, 'E7': 45, 'Eb1': 46, 'Eb2': 47, 'Eb3': 48, 'Eb4': 49, 'Eb5': 50, 'Eb6': 51, 'Eb7': 52, 'F#1': 53, 'F#2': 54, 'F#3': 55, 'F#4': 56, 'F#5': 57, 'F#6': 58, 'F#7': 59, 'F1': 60, 'F2': 61, 'F3': 62, 'F4': 63, 'F5': 64, 'F6': 65, 'F7': 66, 'G#1': 67, 'G#2': 68, 'G#3': 69, 'G#4': 70, 'G#5': 71, 'G#6': 72, 'G1': 73, 'G2': 74, 'G3': 75, 'G4': 76, 'G5': 77, 'G6': 78}


In [6]:
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path) and filename == os.listdir(directory_path)[0]:
        df = pd.read_table(file_path, sep="\t", skiprows=1, names=["noteID", "onset_time", "offset_time", "spelled_pitch", "onset_velocity", "offset_velocity", "channel", "finger_number"])

num_data, num_features = df.shape
x = df.iloc[:, 0:num_features - 1]
y = df.iloc[:, num_features - 1]


# convert "spelled pitch" field to a number
x['spelled_pitch'] = x['spelled_pitch'].map(pitch_to_int_mapping)


x = torch.tensor(x.values.tolist(), dtype=torch.float32)
y = torch.tensor(y.values.astype(float).tolist())
y = y.unsqueeze(1)   # conver from size [289] to size [289, 1]

print(f"x's shape is {x.shape}")
print(f"y's shape is {y.shape}")

x's shape is torch.Size([293, 7])
y's shape is torch.Size([293, 1])


In [7]:
print(f"num features is {num_features} and num data is {num_data}")

num features is 8 and num data is 293


In [8]:
# Pytorch code that does the same least squares fitting, but nn.Module-ized. Iterative regression

class LinearRegressionBaseline(nn.Module):
    def __init__(self, input_dims, output_dims):
        super(LinearRegressionBaseline, self).__init__()
        self.linear = nn.Linear(input_dims, output_dims)

    def forward(self, x):
        return self.linear(x)

In [9]:
my_linear_model = LinearRegressionBaseline(num_features - 1, 1)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(my_linear_model.parameters(), lr=1e-2)    # Needs a VERY small learning rate, else NaN.


# normalize inputs before feeding to model
x = (x - x.mean()) / x.std()
y = (y - y.mean()) / y.std()


print(x.shape)
print(y.shape)
print(my_linear_model(x).shape)


torch.Size([293, 7])
torch.Size([293, 1])
torch.Size([293, 1])


In [10]:

epochs = 5000
for epoch in range(epochs):
    # Forward pass
    y_pred = my_linear_model(x)
    loss = criterion(y_pred, y)
#     print(x.shape)
#     print(y.shape)
#     print(y_pred)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5000 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [5000/5000], Loss: 0.7439


In [11]:
# Check on testing set

x_test = x
y_test = y




# Ensure your model is in evaluation mode
my_linear_model.eval()

# Disable gradient computation for evaluation
with torch.no_grad():
    # Forward pass: Predict on the test set
    y_test_pred = my_linear_model(x_test)

    # Calculate the loss on the test set
    test_loss = criterion(y_test_pred, y_test)

    # Optional: Convert predictions to numpy for further analysis if needed
    y_test_pred_np = y_test_pred.cpu().numpy() if y_test_pred.is_cuda else y_test_pred.numpy()
    y_test_np = y_test.cpu().numpy() if y_test.is_cuda else y_test.numpy()

# Print the results
print("Test Results:")
print(f"Test Loss: {test_loss.item():.4f}")
print("Sample Predictions:")
for i in range(min(5, len(y_test_pred))):  # Display up to 5 predictions
    print(f"Predicted: {y_test_pred_np[i]}, Actual: {y_test_np[i]}")

Test Results:
Test Loss: 0.7438
Sample Predictions:
Predicted: [0.19861197], Actual: [0.17805079]
Predicted: [0.17860365], Actual: [1.4822726]
Predicted: [0.1981008], Actual: [0.17805079]
Predicted: [0.17809272], Actual: [1.1562172]
Predicted: [0.19975638], Actual: [0.17805079]


In [12]:
# Compare with Scikit Normal Equation Result

import numpy as np
from sklearn.linear_model import LinearRegression
import torch

# Assuming x and y are your training tensors
x_np = x.numpy() if isinstance(x, torch.Tensor) else x
y_np = y.numpy() if isinstance(y, torch.Tensor) else y
y_np = y_np.reshape(-1)

print(x_np.shape)
print(y_np.shape)


# Validate with Scikit-Learn
lr = LinearRegression(fit_intercept=True)
lr.fit(x_np, y_np)

# Print Scikit-Learn weights for comparison
print("Weights (Scikit-Learn):", np.hstack([lr.intercept_, lr.coef_]))


x_test_np = x_np
y_test_np = y_np

# Predict on the test set
y_test_pred = lr.predict(x_test_np)

# Compute Mean Squared Error (MSE) loss
test_loss = np.mean((y_test_pred - y_test_np) ** 2)

# Print the loss
print("Test Loss (Scikit-Learn):", test_loss)    # About the same loss: 0.53!

(293, 7)
(293,)
Weights (Scikit-Learn): [ -87.728645      0.22731297   12.838359    -15.811432      0.13658929
    0.22497249    0.22756538 -104.933     ]
Test Loss (Scikit-Learn): 0.233975


# Idea 2: Add more layers

In [13]:
# deep neural net with more layers

class DeepNeuralNet(nn.Module):
    def __init__(self, input_dims, output_dims):
        super(DeepNeuralNet, self).__init__()
        self.mlp_layer = nn.Sequential(
            nn.Linear(input_dims, 4 * input_dims),
            nn.ReLU(),
            nn.Linear(4 * input_dims, output_dims),
        )


    def forward(self, x):
        return self.mlp_layer(x)

In [14]:
dnn_model = DeepNeuralNet(num_features - 1, 1)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(dnn_model.parameters(), lr=1e-2)    # Needs a VERY small learning rate, else NaN.


# normalize inputs before feeding to model
x = (x - x.mean()) / x.std()
y = (y - y.mean()) / y.std()


print(x.shape)
print(y.shape)
print(dnn_model(x).shape)


# Adding more layers helps a lot!

epochs = 1000
for epoch in range(epochs):
    # Forward pass
    y_pred = dnn_model(x)
    loss = criterion(y_pred, y)
#     print(x.shape)
#     print(y.shape)
#     print(y_pred)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5000 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

torch.Size([293, 7])
torch.Size([293, 1])
torch.Size([293, 1])


# Testing code

Testing on some random other piece

In [15]:
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path) and filename == os.listdir(directory_path)[1]:
        test_df = pd.read_table(file_path, sep="\t", skiprows=1, names=["noteID", "onset_time", "offset_time", "spelled_pitch", "onset_velocity", "offset_velocity", "channel", "finger_number"])
        print(filename)

x_test = test_df.iloc[:, 0:num_features - 1]
y_test = test_df.iloc[:, num_features - 1]

x_test['spelled_pitch'] = x_test['spelled_pitch'].map(pitch_to_int_mapping)


# print(x_test)

# NORMALIZE FIRST
# Normalize x_test
for col in x_test.columns:
    std = x_test[col].std()
    if std == 0:  # Check if all values are the same
        x_test[col] = 0  # Assign all entries in this column to 0
    else:
        x_test[col] = (x_test[col] - x_test[col].mean()) / std

# Normalize y_test
y_test_std = y_test.std()
if y_test_std == 0:  # Check if all values are the same
    y_test = 0  # Assign all entries in y_test to 0
else:
    y_test = (y_test - y_test.mean()) / y_test_std



x_test = torch.tensor(x_test.values.tolist(), dtype=torch.float32)
y_test = torch.tensor(y_test.values.astype(float).tolist())
y_test = y_test.unsqueeze(1)   # conver from size [289] to size [289, 1]


# print(y_test)


# print(x_test.isna().sum())
# print(y_test.shape)

002-5_fingering.txt


In [16]:

# Check on testing set

# x_test = x
# y_test = y

# print(x_test)
# print(y_test)


# Ensure your model is in evaluation mode
dnn_model.eval()

# Disable gradient computation for evaluation
with torch.no_grad():
    # Forward pass: Predict on the test set
    y_test_pred = dnn_model(x_test)

    # Calculate the loss on the test set
    test_loss = criterion(y_test_pred, y_test)

    # Optional: Convert predictions to numpy for further analysis if needed
    y_test_pred_np = y_test_pred.cpu().numpy() if y_test_pred.is_cuda else y_test_pred.numpy()
    y_test_np = y_test.cpu().numpy() if y_test.is_cuda else y_test.numpy()

# Print the results
print("Test Results:")
print(f"Test Loss: {test_loss.item():.4f}")
print("Sample Predictions:")
for i in range(min(5, len(y_test_pred))):  # Display up to 5 predictions
    print(f"Predicted: {y_test_pred_np[i]}, Actual: {y_test_np[i]}")

Test Results:
Test Loss: 2091.3459
Sample Predictions:
Predicted: [-14.536429], Actual: [0.3052198]
Predicted: [-24.420633], Actual: [0.64844805]
Predicted: [-13.382349], Actual: [0.3052198]
Predicted: [-14.38389], Actual: [0.99167633]
Predicted: [-12.240117], Actual: [0.3052198]


In [28]:
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

# Step 1: Load the specific file
specific_file = os.listdir(directory_path)[10]  # Replace 10 with the desired index
file_path = os.path.join(directory_path, specific_file)

my_df = pd.DataFrame()

if os.path.isfile(file_path):
    print(f"ANNOTATING FILE: {specific_file}")
    my_df = pd.read_table(
        file_path,
        sep="\t",
        skiprows=1,
        names=["noteID", "onset_time", "offset_time", "spelled_pitch", "onset_velocity", "offset_velocity", "channel", "finger_number"]
    )


column_name = "finger_number"  # Replace with the actual column name
if not pd.api.types.is_integer_dtype(my_df[column_name]):
    my_df.iloc[:, -1] = my_df.iloc[:, -1].where(
        ~my_df.iloc[:, num_features - 1].str.contains('_', na=False),
        my_df.iloc[:, num_features - 1].str.split('_').str[0]
    ).astype(int)




# Step 2: Preprocess the data
# Normalize and map spelled_pitch
my_df['spelled_pitch'] = my_df['spelled_pitch'].map(pitch_to_int_mapping)

features = my_df.iloc[:, :-1]
labels = my_df.iloc[:, -1].astype(int) + 5  # Adjust labels if necessary for CrossEntropyLoss



# Replace NaN or invalid mappings if needed
# features.fillna(0, inplace=True)


# Normalize features: CAN'T JUST DO MEAN / STD SINCE STD MIGHT BE 0!!!
for col in features.columns:
    std = features[col].std()
    if std == 0:  # Check if all values are the same
        features[col] = 0  # Assign all entries in this column to 0
    else:
        features[col] = (features[col] - features[col].mean()) / std









# Convert to PyTorch tensors
x_piece = torch.tensor(features.values, dtype=torch.float32).unsqueeze(0)  # Shape: [1, seq_len, num_features]
y_piece = torch.tensor(labels.values, dtype=int).unsqueeze(0)  # Shape: [1, seq_len]



# Step 3: Create a DataLoader
piece_dataset = TensorDataset(x_piece, y_piece)
piece_loader = DataLoader(piece_dataset, batch_size=1, shuffle=False)

# Step 4: Predict the fingering sequence
model.eval()
predicted_fingering = []

with torch.no_grad():
    for x_batch, _ in piece_loader:
        x_batch = x_batch.to(device)
        outputs = model(x_batch)  # Shape: [1, seq_len, num_classes]
        preds = torch.argmax(outputs, dim=-1)  # Shape: [1, seq_len]
        predicted_fingering = preds.squeeze(0).tolist()

# Step 5: Output the predictions
print(f"Predicted Fingering Sequence for {specific_file}:")
print(predicted_fingering)


# print out actual
print(f"Actual Fingering:")
print(labels.tolist())


# print out overall "loss"


ANNOTATING FILE: 053-1_fingering.txt
Predicted Fingering Sequence for 053-1_fingering.txt:
[8, 0, 4, 4, 2, 2, 2, 4, 4, 2, 4, 6, 0, 2, 10, 0, 2, 6, 0, 2, 6, 0, 2, 6, 0, 2, 6, 8, 0, 4, 6, 8, 0, 4, 6, 8, 0, 4, 6, 8, 0, 4, 6, 8, 0, 4, 6, 8, 0, 4, 6, 8, 0, 4, 4, 2, 2, 2, 4, 4, 2, 4, 6, 0, 2, 10, 0, 2, 6, 0, 2, 7, 0, 2, 7, 0, 2, 6, 8, 0, 4, 6, 8, 0, 4, 6, 8, 0, 4, 6, 8, 0, 4, 6, 8, 0, 4, 6, 8, 0, 4, 6, 8, 0, 4, 4, 2, 8, 8, 0, 10, 8, 0, 10, 8, 10, 10, 10, 10, 10, 10, 0, 4, 6, 8, 0, 10, 8, 0, 10, 8, 8, 10, 10, 10, 10, 10, 0, 4, 4, 2, 7, 8, 0, 4, 6, 8, 0, 4, 6, 8, 10, 10, 10, 10, 0, 4, 6, 8, 0, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, 4, 6, 8, 0, 10, 8, 8, 10, 10, 10, 10, 10, 0, 4, 4, 2, 2, 4, 4, 4, 2, 8, 8, 10, 10, 10, 10, 0, 4, 4, 6, 0, 6, 8, 0, 10, 0, 6, 4, 6, 4, 6, 4, 4, 4, 6, 8, 10, 10, 10, 10, 0, 4, 4, 6, 0, 6, 8, 0, 10, 0, 6, 4, 6, 4, 6, 4, 4, 4]
Actual Fingering:
[8, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 8, 1, 3, 6, 1, 3, 7, 1, 3, 6, 1, 4, 7, 1, 4, 8, 6, 1, 4, 7, 8, 1, 4, 6, 

In [29]:
# print metrics for a specific piece
labels = labels.tolist()

# Calculate model loss and accuracy
def calc_metrics(preds, actuals, criterion):
    """
    Calculate the Cross-Entropy loss and accuracy for the predicted vs. actual labels.
    """

    num_wrong = 0

    if len(preds) != len(actuals):
        print("preds and actuals not of same size")
        return

    for i, value in enumerate(preds):
        if value != actuals[i]:
            num_wrong += 1
            print(f"error at pos {i}: note is {value} but should be {preds[i]}")

    print(f"num_wrong is {num_wrong}")
    print(f"len(preds) is {len(preds)}")

    accuracy = 1 - (num_wrong / len(preds))
    return accuracy

# print(predicted_fingering)
# print(labels.tolist())
# Calculate overall loss and accuracy
overall_accuracy = calc_metrics(predicted_fingering, labels, criterion)

# Print the overall loss and accuracy
# print(f"Overall Loss: {overall_loss:.4f}")
print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")

error at pos 1: note is 0 but should be 0
error at pos 2: note is 4 but should be 4
error at pos 3: note is 4 but should be 4
error at pos 4: note is 2 but should be 2
error at pos 5: note is 2 but should be 2
error at pos 6: note is 2 but should be 2
error at pos 7: note is 4 but should be 4
error at pos 8: note is 4 but should be 4
error at pos 9: note is 2 but should be 2
error at pos 10: note is 4 but should be 4
error at pos 11: note is 6 but should be 6
error at pos 12: note is 0 but should be 0
error at pos 13: note is 2 but should be 2
error at pos 14: note is 10 but should be 10
error at pos 15: note is 0 but should be 0
error at pos 16: note is 2 but should be 2
error at pos 17: note is 6 but should be 6
error at pos 18: note is 0 but should be 0
error at pos 19: note is 2 but should be 2
error at pos 21: note is 0 but should be 0
error at pos 22: note is 2 but should be 2
error at pos 23: note is 6 but should be 6
error at pos 24: note is 0 but should be 0
error at pos 25: n

# Testing if Chinese reward model works

In [32]:
from reward import PianoFingeringModel, Hand, Finger, Note, NoteFingerPair
reward_model = PianoFingeringModel()

# ADD CODE HERE THAT TESTS THE REWARD MODEL.
# Specifically, I want to essentially run the reward signal over the (probably bad)
# linear regression fingering predictions and the ground truth fingering.
# I want to be able to tell if the reward model is working as expected if it assigns high
# reward to the ground truth and low reward to the bad predictions